<a href="https://colab.research.google.com/github/HyelinNAM/TIL/blob/master/0206_TFIDF%26Cosine_Similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

link ='https://drive.google.com/open?id=1YXyWrZkuM_3mcNd3UMui3Hp2aQuSV03m'
fluff, id = link.split('=')

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile("movies_metadata.csv")  
data = pd.read_csv("movies_metadata.csv")

In [0]:
data.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [0]:
data.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [0]:
data = data.head(20000)

In [0]:
data['overview'].isnull() # Null 값 체크

0        False
1        False
2        False
3        False
4        False
         ...  
19995    False
19996    False
19997    False
19998    False
19999    False
Name: overview, Length: 20000, dtype: bool

In [0]:
data['overview'] = data['overview'].fillna('')

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')

In [0]:
tfidf

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [0]:
tfidf_matrix = tfidf.fit_transform(data['overview'])

In [0]:
tfidf_ar = tfidf_matrix.toarray()
print(type(tfidf_ar))

<class 'numpy.ndarray'>


In [0]:
import numpy as np

np.set_printoptions(threshold=10)

In [0]:
print(type(tfidf_matrix))
print(tfidf_matrix.shape)
print(tfidf_matrix)

<class 'scipy.sparse.csr.csr_matrix'>
(20000, 47487)
  (0, 11645)	0.13942792746208266
  (0, 2811)	0.1524688231743182
  (0, 24325)	0.10272285889754121
  (0, 14393)	0.10323587624822204
  (0, 12920)	0.13359137839416452
  (0, 30780)	0.10362949000778948
  (0, 37778)	0.13329592929044795
  (0, 7974)	0.13030575543855816
  (0, 32424)	0.13359137839416452
  (0, 19047)	0.10485920860731182
  (0, 32291)	0.09637786759647918
  (0, 25229)	0.12052668500637997
  (0, 1254)	0.1489611323980068
  (0, 37166)	0.11699407550916606
  (0, 24759)	0.19553422928903877
  (0, 6290)	0.5036643709879097
  (0, 5775)	0.10961099833635705
  (0, 4736)	0.12600776612424489
  (0, 36185)	0.11514050233158798
  (0, 18727)	0.13359137839416452
  (0, 24957)	0.09146351035625488
  (0, 43131)	0.15491381382108724
  (0, 2051)	0.4159373110833513
  (0, 46617)	0.46474144146326174
  (0, 24361)	0.10943293956646011
  :	:
  (19997, 13221)	0.17897321248705433
  (19997, 37592)	0.15166031585027698
  (19997, 45919)	0.1821847673691749
  (19997, 46772)	

In [0]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix,tfidf_matrix)

In [0]:
type(cosine_sim)

numpy.ndarray

In [0]:
import pandas as pd

indices = pd.Series(data.index, index=data['title'])
print(indices.head())

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64


In [0]:
idx = indices['Jumanji']
print(idx)

1


In [0]:
def get_recommendations(title, cosine_sim=cosine_sim):
  idx = indices[title]

  sim_scores = list(enumerate(cosine_sim[idx]))
  sim_scores = sorted(sim_scores,key=lambda x: x[1],reverse=True)
  sim_scores = sim_scores[1:11]
  movie_indices = [i[0] for i in sim_scores]

  return data['title'].iloc[movie_indices]

In [0]:
get_recommendations('Batman Forever')

18252                      The Dark Knight Rises
19792    Batman: The Dark Knight Returns, Part 1
12481                            The Dark Knight
3095                Batman: Mask of the Phantasm
10122                              Batman Begins
1328                              Batman Returns
15511                 Batman: Under the Red Hood
9230          Batman Beyond: Return of the Joker
1491                              Batman & Robin
18035                           Batman: Year One
Name: title, dtype: object